In [43]:
from bottle import route, run, get, post, request
import bson
import os
import json
from dotenv import load_dotenv
load_dotenv()
from src.mongo import connectCollection
from src.sentiment import sentimentAnalyzer

In [2]:
db, coll = connectCollection('chats','messages_linked')

In [7]:
chat_id = '0'
data = list(coll.find({'idChat': int(chat_id)}))

In [35]:
user=0
message='hahaha'

In [51]:
chat_id = '6'
db2, coll2 = connectCollection('chats','chats')
chat = list(coll2.find({'idChat': int(chat_id)}))

In [52]:
len(chat)

0

In [20]:
import re
chatUsers = [int(e) for e in re.sub('\[|\]','',chat[0]['users']).split(', ')]

In [24]:
if user not in chatUsers:
    print('no')

In [32]:
db3, coll3 = connectCollection('chats','users')
selectedUser = list(coll3.find({'idUser': user}))

'John Wick'

In [37]:
import datetime

newMessage = {'datetime': datetime.datetime.utcnow(),
                'idChat': int(chat_id),
                'idMessage': max([e['idMessage'] for e in data])+1,
                'idUser': user,
                'text': message,
                'userName': selectedUser[0]['userName'],
                'user_id': selectedUser[0]['_id']}
message_id = coll.insert_one(newMessage).inserted_id

In [38]:
message_id

ObjectId('5de3122c0f60829b1b821bf2')

In [39]:
{'ObjectId': str(message_id), 'MessageId': newMessage['idMessage']}

{'ObjectId': '5de3122c0f60829b1b821bf2', 'MessageId': 10}